## The Challenge

Many investment firms are working with a complex patchwork of legacy systems that have been introduced and added to over many years, as a result of mergers or to support specific business areas and / or asset classes. 

Each system has its own data model, frequency of updates and idiosyncrasies, making it challenging to provide an aggregated view of all your investment portfolios.

## The Solution

LUSID is an open platform and is API driven, letting you connect both source and downstream systems. LUSID provides a flexible way to load your existing data in without significant data mapping, for example transaction types and instrument modelling. This allows existing users of the data to be repointed to LUSID with no risk of being affected by the migration, and take advantage of new capabilities in a gradual and controlled way.

To demonstrate this capability you will migrate a single fund from multiple source systems into LUSID. You have three systems to migrate across, each system specialises in a different asset class. You have a system for bonds, a system for equities and a system for options. To achieve this migration you will:

1) Create your instrument universe using a range of identifires

2) Set up a segregated scope for each source system

3) Create a portfolio for your fund in each of your scopes

4) Set your initial holdings

5) Set your transaction type configuration

6) Load a day's worth of transactions

7) Attempt to get your updated holdings

8) Update your transaction type configuration & try again to get your updated holdings

9) Load your end of day positions

10) Perform a bi-temporal reconciliation to ensure there are no breaks

11) Group the views from each system together to create an overall view of your fund

12) Get the value of your fund

First things first though import the libraries that you need and authenticate your LUSID client.

*Run the cell below to import the libraries and authenticate your LUSID client*

In [1]:
# Import LUSID
import lusid.models as models
import lusid
import lusid_sample_data as import_data

# Import Libraries
import pprint
from datetime import datetime, timedelta, time
import pytz
import printer as prettyprint
import pandas as pd
import numpy as np
import json
import uuid

# Authenticate our user and create our API client
client = import_data.authenticate_secrets()

print ('LUSID Environment Initialised')
print ('LUSID SDK Version: ', client.metadata.get_lusid_versions().build_version)

LUSID Environment Initialised
LUSID SDK Version:  0.5.2521.0


Your set up now looks like the below. As you progress through the migration you will see updated diagrams of the state of the system.

![Init](img/multiplesystems-initialstate.gif)

## 1) Create your Instrument Universe

Before you can take on any holdings or make any trades you need to ensure that your instrument universe has been populated. In this case you will import your instrument universe from a number of CSV files, one for each source system. Read more about instruments in LUSID in the [LUSID Knowledge Base: Instruments](https://support.lusid.com/what-is-an-instrument).

*Run the cells below to import your instrument universe from your source systems*

In [53]:
equity_instruments = pd.read_csv('data/multiplesystems-instruments-equities.csv')
equity_instruments.head()

,InstrumentName,ClientInternal,Currency,Isin,Figi,ExchangeCode,CountryIssue,Ticker,MarketSector,SecurityType,Coupon
0,Amazon_Nasdaq_AMZN,imd_34634534,USD,US0231351067,BBG000BVPXP1,UN,united_states_america,AMZN,equity,common_stock,NaN
1,Apple_Nasdaq_AAPL,imd_35345345,USD,US0378331005,BBG000B9XVV8,UN,united_states_america,AAPL,equity,common_stock,NaN
2,BP_LondonStockEx_BP,imd_43535553,GBP,GB0007980591,BBG000C05BD1,LN,united_kingdom,BP/,equity,common_stock,NaN
3,BurfordCapital_LondonStockEx_BUR,imd_43534356,GBP,GG00B4L84979,BBG000PN88Q7,LN,united_kingdom,BUR,equity,common_stock,NaN
4,EKFDiagnostics_LondonStockEx_EKF,imd_34535355,GBP,GB0031509804,BBG000BVNBN3,LN,united_kingdom,EKF,equity,common_stock,NaN


In [3]:
bond_instruments = pd.read_csv('data/multiplesystems-instruments-bonds.csv')
bond_instruments.head()

,InstrumentName,ClientInternal,Currency,Isin,Figi,ExchangeCode,CountryIssue,Ticker,MarketSector,SecurityType,Coupon
0,UKGiltTreasury_2.0_2025,imd_34534536,GBP,GB00BTHH2R79,NaN,LN,united_kingdom,UKT 2 09/07/25,govt,uk_gilt_stock,2.000
1,UKGiltTreasury_3.5_2045,imd_54234532,GBP,GB00BN65R313,NaN,LN,united_kingdom,UKT 3.5 01/22/45,govt,uk_gilt_stock,3.500
2,UKGiltTreasury_3.75_2021,imd_34643653,GBP,GB00B4RMG977,NaN,LN,united_kingdom,UKT 3.75 09/07/21,govt,uk_gilt_stock,3.750
3,UKGiltTreasury_4.5_2034,imd_34534534,GBP,GB00B52WS153,NaN,LN,united_kingdom,UKT 4.5 09/07/34,govt,uk_gilt_stock,4.500
4,USTreasury_2.00_2021,imd_34535347,USD,US912828U816,NaN,BERLIN,united_states_america,T 2 12/31/21,govt,us_government,2.000
5,USTreasury_6.875_2025,imd_34534539,USD,US912810EV62,NaN,NEW YORK,united_states_america,T 6.875 08/15/25,govt,us_government,6.875


In [4]:
option_instruments = pd.read_csv('data/multiplesystems-instruments-options.csv')
option_instruments.head()

,InstrumentName,ClientInternal,Currency,Isin,Figi,FigiComposite,Sedol,ExchangeCode,CountryIssue,Ticker,MarketSector,SecurityType,Coupon
0,October 19 Calls on AMZN US,imd_84634539,USD,NaN,BBG00NFXK409,BBG00NFXK409,NaN,UN,united_states_america,AMZN 10/18/19 C1365,equity,equity_option,NaN
1,October 19 Puts on AAPL US,imd_85345347,USD,NaN,BBG00NBRV912,BBG00NBRV912,NaN,UN,united_states_america,AAPL 10/18/19 P140,equity,equity_option,NaN
2,September 19 Calls on BP/ LN,imd_83535553,GBP,NaN,BBG00M2Z8958,NaN,NaN,LN,united_kingdom,BPA 09/20/19 C570,equity,equity_option,NaN


Now that you have the details for your instruments you can go ahead and create an instrument definition for each instrument. These can then be upserted into LUSID. Read about instrument definitions here [LUSID Knowledge Base: What is an Instrument?](https://support.lusid.com/what-is-an-instrument).

As part of this definition you will attach both unique and non-unique identifiers to your instruments. Read more about identifiers here [LUSID Knowledge Base: Which Instrument Identifier Scheme Should I Use?](https://support.lusid.com/which-instrument-identifier-schemes-should-i-use-with-lusid).

You use an upsert method to add instrument definitions to the instrument universe in LUSID. Read more about the behaviour of the upsert method here [LUSID Knowledge Base: Upsert](https://support.lusid.com/upsert-command).

For further usage of the upsert instruments API call refer to the [LUSID API Docs: Upserting Instruments](https://docs.lusid.com/#operation/UpsertInstruments).

As each source system uses different identifiers for the instruments you will need to create a function which allows you to easily select the identifiers that you would like to use in the instrument definitions for the instruments from each source system.

*Run the cell below to create a function for upserting instruments from any one of your sources*

In [5]:
def upsert_instruments(instrument_universe=None, identifier_columns=None):
    """
    This function upserts instruments from a dataframe 
    
    Keyword arguments:
    instrument_universe (Pandas DataFrame) -- The imported instrument universe
    identifer_columns (Tuple (string, string)) -- The identifier name in CSV file 
    and identifier name in LUSID
    
    Returns:
    N/A
    """
    
    # Initialise your batch upsert request
    batch_upsert_request = {}

    # Using your instrument universe create your batch request
    for index, instrument in instrument_universe.iterrows():
    
        # Create your identifiers
        identifiers = {}
        for identifier in identifier_columns:
            identifiers[identifier[1]] = models.InstrumentIdValue(
                value=instrument[identifier[0]])

        # Build your request and add it to the dictionary
        batch_upsert_request[instrument['InstrumentName']] = models.InstrumentDefinition(
            name=instrument['InstrumentName'],
            identifiers=identifiers
        )
        
    # Call LUSID to upsert your instrument defintions
    instrument_response = client.instruments.upsert_instruments(
        request_body=batch_upsert_request)
    
    # Pretty print the response
    prettyprint.instrument_response(instrument_response)

Now that you have a function to upsert your instruments you can upsert the instruments from each source.

For bonds you have access to an ISIN, client internal identifier and a ticker. 

*Run the cell below to upsert your bond instruments into LUSID*

In [6]:
# Upsert your bond instruments into LUSID
upsert_instruments(
    instrument_universe=bond_instruments, 
    identifier_columns=[
        ('ClientInternal', 'ClientInternal'),
        ('Isin', 'Isin'),
        ('Ticker', 'Ticker')
    ])

Instrument Successfully Upserted: UKGiltTreasury_2.0_2025
ClientInternal ID: imd_34534536
LUSID Instrument ID: LUID_05Z3KLMD


Instrument Successfully Upserted: UKGiltTreasury_3.5_2045
ClientInternal ID: imd_54234532
LUSID Instrument ID: LUID_G54PWWXO


Instrument Successfully Upserted: USTreasury_2.00_2021
ClientInternal ID: imd_34535347
LUSID Instrument ID: LUID_CZ7TP760


Instrument Successfully Upserted: USTreasury_6.875_2025
ClientInternal ID: imd_34534539
LUSID Instrument ID: LUID_8YCAWIXB


Instrument Successfully Upserted: UKGiltTreasury_3.75_2021
ClientInternal ID: imd_34643653
LUSID Instrument ID: LUID_Z35EE199


Instrument Successfully Upserted: UKGiltTreasury_4.5_2034
ClientInternal ID: imd_34534534
LUSID Instrument ID: LUID_3ZHAZR8Q


6  instruments upserted successfully
0  instrument upsert failures


For equities you have access to an ISIN, FIGI, client internal identifier and a ticker.

*Run the cell below to upsert your equity instruments into LUSID*

In [7]:
# Upsert your equity instruments into LUSID
upsert_instruments(
    instrument_universe=equity_instruments, 
    identifier_columns=[
        ('ClientInternal', 'ClientInternal'),
        ('Isin', 'Isin'),
        ('Figi', 'Figi'),
        ('Ticker', 'Ticker')
    ])

Instrument Successfully Upserted: BurfordCapital_LondonStockEx_BUR
ClientInternal ID: imd_43534356
LUSID Instrument ID: LUID_P1739DDC


Instrument Successfully Upserted: Apple_Nasdaq_AAPL
ClientInternal ID: imd_35345345
LUSID Instrument ID: LUID_HS1DA84K


Instrument Successfully Upserted: BP_LondonStockEx_BP
ClientInternal ID: imd_43535553
LUSID Instrument ID: LUID_LDYURLNU


Instrument Successfully Upserted: Glencore_LondonStockEx_GLEN
ClientInternal ID: imd_34534555
LUSID Instrument ID: LUID_S6N347I3


Instrument Successfully Upserted: Kingfisher_LondonStockEx_KGF
ClientInternal ID: imd_34535552
LUSID Instrument ID: LUID_KHHWEILW


Instrument Successfully Upserted: WPP_LondonStockEx_WPP
ClientInternal ID: imd_34536734
LUSID Instrument ID: LUID_46W54EOB


Instrument Successfully Upserted: Whitebread_LondonStockEx_WTB
ClientInternal ID: imd_35349900
LUSID Instrument ID: LUID_7RXE0ELE


Instrument Successfully Upserted: JustEat_LondonStockEx_JE
ClientInternal ID: imd_35436366
LUSID Ins

For options you have access to a FIGI, client internal identifier and a ticker.

*Run the cell below to upsert your option instruments into LUSID*

In [8]:
# Upsert your option instruments into LUSID
upsert_instruments(
    instrument_universe=option_instruments, 
    identifier_columns=[
        ('ClientInternal', 'ClientInternal'),
        ('Figi', 'Figi'),
        ('Ticker', 'Ticker')
    ])

Instrument Successfully Upserted: October 19 Puts on AAPL US
ClientInternal ID: imd_85345347
LUSID Instrument ID: LUID_9U1W78BB


Instrument Successfully Upserted: September 19 Calls on BP/ LN
ClientInternal ID: imd_83535553
LUSID Instrument ID: LUID_FR4JL070


Instrument Successfully Upserted: October 19 Calls on AMZN US
ClientInternal ID: imd_84634539
LUSID Instrument ID: LUID_HTNT5VFB


3  instruments upserted successfully
0  instrument upsert failures


You have now upserted all of your instruments into the LUSID instrument master.

![Init](img/multiplesystems-instrumentmaster.gif)

## 2) Set up a Scope for each Source System

To segregate the position and transaction data from each source system you can use scopes. Read more about scopes here [LUSID Knowledge Base: Scopes](https://support.lusid.com/what-is-a-scope-in-lusid-and-how-is-it-used).

You will create a scope for each source system.

*Run the cell below to create a scope for each system*

In [9]:
bonds_scope = 'bonds_system_{}'.format(str(uuid.uuid4())[:4])
equities_scope = 'equities_system_{}'.format(str(uuid.uuid4())[:4])
options_scope = 'options_system_{}'.format(str(uuid.uuid4())[:4])

scopes = [bonds_scope, equities_scope, options_scope]

prettyprint.heading('Bonds Scope', bonds_scope)
prettyprint.heading('Equities Scope', bonds_scope)
prettyprint.heading('Options Scope', bonds_scope)

Bonds Scope: bonds_system_32cc
Equities Scope: bonds_system_32cc
Options Scope: bonds_system_32cc


![Init](img/multiplesystems-scopes.gif)

## 3) Create your Portfolio in Each Scope

To hold the positions and transactions of the fund you need to create a portfolio in each scope.

Note that every portfolio can be referenced by a unique code. Read more about portfolios in the [LUSID Knowledge Base: Portfolios](https://support.lusid.com/what-is-2).

For further usage of the create portfolio API call refer to the [LUSID API Docs: Create Portfolio](https://docs.lusid.com/#operation/CreatePortfolio).

Note that when you create the portolios in the cell below you are creating it with a 'created' date of 1052 days ago. This number is rather arbitary, in practice it should be the date the portfolio came into existence regardless of the system you first created it in, read more about the importance of the created date on a portfolio in the [LUSID Knowledge Base: Importance of Portfolio Creation Date](https://support.lusid.com/importance-of-portfolio-creation-date).

*Run the cell below to create your portfolios*

In [55]:
# Set the code of your portfolio
portfolio_code = str(uuid.uuid4())

# Set the creation date of your portfolio 
portfolio_creation_date = datetime.now(pytz.UTC) - timedelta(days=1052)

# Build your request to create your portfolio
request = models.CreateTransactionPortfolioRequest(
    display_name='Global Strategies Fund',
    code=portfolio_code,
    base_currency='USD',
    description='The global strategies fund',
    created=portfolio_creation_date,
    corporate_action_source_id=None,
    accounting_method='AverageCost',
    sub_holding_keys=None,
    properties=None)

# Iterate over the scopes for the source systems 
for scope in scopes:
    
    # Call LUSID to create your portfolio
    response = client.transaction_portfolios.create_portfolio(
        scope=scope,
        create_transaction_portfolio_request=request)

    # Pretty print the response
    prettyprint.portfolio_response(response)

Portfolio Created
Scope: bonds_system_32cc
Code: c480c32e-708f-4bca-bb74-321956e26c95
Portfolio Effective From: 2016-05-31 13:52:36.912494+00:00
Portfolio Created On: 2019-04-18 13:52:35.514074+00:00

Portfolio Created
Scope: equities_system_d93d
Code: c480c32e-708f-4bca-bb74-321956e26c95
Portfolio Effective From: 2016-05-31 13:52:36.912494+00:00
Portfolio Created On: 2019-04-18 13:52:35.806672+00:00

Portfolio Created
Scope: options_system_8f3d
Code: c480c32e-708f-4bca-bb74-321956e26c95
Portfolio Effective From: 2016-05-31 13:52:36.912494+00:00
Portfolio Created On: 2019-04-18 13:52:36.023668+00:00



You now have an empty portfolio inside each scope.

![Init](img/multiplesystems-portfolios.gif)

## 4) Set your Initial Holdings

Now that you have your instrument universe populated and portfolios created you can load your initial holdings into your portfolio. In this case you will also import your holdings from CSV files. 

*Run the cells below to import your holdings*

In [11]:
bond_holdings = pd.read_csv('data/multiplesystems-holdings-bonds.csv')
bond_holdings.head()

,PortfolioCode,InstrumentName,Quantity,Price,Currency,Figi,ClientInternal
0,Global-Strategies,UKGiltTreasury_2.0_2025,405589,106.637,GBP,NaN,imd_34534536
1,Global-Strategies,UKGiltTreasury_3.5_2045,266169,134.433,GBP,NaN,imd_54234532
2,Global-Strategies,UKGiltTreasury_3.75_2021,661713,108.126,GBP,NaN,imd_34643653
3,Global-Strategies,UKGiltTreasury_4.5_2034,77481,140.572,GBP,NaN,imd_34534534
4,Global-Strategies,USTreasury_2.00_2021,1440244,97.900,USD,NaN,imd_34535347


In [12]:
equity_holdings = pd.read_csv('data/multiplesystems-holdings-equities.csv')
equity_holdings.head()

,PortfolioCode,InstrumentName,Quantity,Price,Currency,Figi
0,Global-Strategies,GBP_Cash,5557333,1.000,GBP,NaN
1,Global-Strategies,Glencore_LondonStockEx_GLEN,905141,2.762,GBP,BBG001MM1KV4
2,Global-Strategies,Kingfisher_LondonStockEx_KGF,1362038,2.276,GBP,BBG000BKH1W6
3,Global-Strategies,BurfordCapital_LondonStockEx_BUR,853486,14.060,GBP,BBG000PN88Q7
4,Global-Strategies,EKFDiagnostics_LondonStockEx_EKF,925925,0.270,GBP,BBG000BVNBN3


In [13]:
option_holdings = pd.read_csv('data/multiplesystems-holdings-options.csv')
option_holdings.head()

,PortfolioCode,InstrumentName,Quantity,Price,Currency,Figi
0,Global-Strategies,October 19 Calls on AMZN US,100,245.28,USD,BBG00NFXK409
1,Global-Strategies,October 19 Puts on AAPL US,150,383.00,USD,BBG00NBRV912
2,Global-Strategies,September 19 Calls on BP/ LN,124,140.00,GBP,BBG00M2Z8958


As each source system uses different identifiers for the holdings you will need to create a function which allows you to easily select the identifiers that you would like to use in the identifying the instrument related to each holding.

Read more about how making an adjustment or setting the holdings on a portfolio affects it here [LUSID Knowledge Base: The effect of holding adjustments](https://support.lusid.com/how-do-holding-adjustments-affect-a-portfolio).

For further usage of the set holdings API call refer to the [LUSID API Docs: Set Holdings](https://docs.lusid.com/#operation/SetHoldings).

*Run the cell below to create a function to handle adding your holdings*

In [14]:
def load_holdings(holdings, scope, code, holdings_effective_date, instrument_identifier):
    """
    This function takes a set of holdings from a CSV file and sets them on the appropriate
    portfolio
    
    Keyword arguments:
    holdings (Pandas DataFrame) -- The imported holdings
    scope (string) -- The scope of the portfolio to set the holdings on
    code (string) -- The code of the portfolio to set the holdings on
    holdings_effective_date (datetime) - The timezone aware datetime from which the
    holdings should be effective
    instrument_identifier (string) -- The name of the identifier to use to resolve
    each holding to an instrument in the instrument master
    
    Returns:
    N/A
    """
    
    # Iterate the portfolios in the holdings CSV, note in this case you only have one
    for portfolio in holdings['PortfolioCode'].unique():
        # Initialise a list to hold your adjustments
        holding_adjustments = []

        # Iterate over the holdings in each portfolio
        for index, holding in holdings.loc[
            holdings['PortfolioCode'] == portfolio].iterrows():

            # Set your instrument identifiers based on whether or not instrument is cash
            if 'Cash' in holding['InstrumentName']:
                identifier_key = 'Instrument/default/Currency'
                identifier = holding['InstrumentName'].split('_')[0]
            else:
                identifier_key = 'Instrument/default/{}'.format(instrument_identifier)
                identifier = holding[instrument_identifier]

            # Create your holding adjustment and append it to your list
            holding_adjustments.append(
                models.AdjustHoldingRequest(
                    instrument_identifiers={
                        identifier_key: identifier},
                    tax_lots=[
                        models.TargetTaxLotRequest(
                            units=holding['Quantity'],
                            cost=models.CurrencyAndAmount(
                                amount=holding['Quantity'] * holding['Price'],
                                currency=holding['Currency']),
                            portfolio_cost=holding['Quantity'] * holding['Price'],
                            price=holding['Price'])
                    ]
                )
            )

        # Call LUSID to set your initial holdings
        response = client.transaction_portfolios.set_holdings(
            scope=scope,
            code=code,
            effective_at=holdings_effective_date,
            adjust_holding_request=holding_adjustments)

        # Pretty print our response from LUSID
        prettyprint.set_holdings_response(response, scope, portfolio)

Now that you have a function to set your holdings you can set them for each source. 

These holdings will be effective as of 2 days ago. 

For bonds you will use the internal client identifier. 

Run the cell below to set your bond holdings for your portfolio.

*Run the cell below to set your bond holdings in LUSID*

In [15]:
# Make the holdings effective from two days ago
holdings_effective_date = datetime.now(pytz.UTC) - timedelta(days=2)

load_holdings(
    holdings=bond_holdings, 
    scope=bonds_scope, 
    code=portfolio_code, 
    holdings_effective_date=holdings_effective_date, 
    instrument_identifier='ClientInternal')

Holdings Successfully Set for Portfolio
Scope: bonds_system_32cc
Code: Global-Strategies
Holdings Effective From: 2019-04-16 13:43:54.639262+00:00
Holdings Created On: 2019-04-18 13:43:54.940053+00:00



For equities you will use the FIGI identifier.

Run the cell below to set your equity holdings for your portfolio.

*Run the cell below to set your equity holdings in LUSID*

In [16]:
load_holdings(
    holdings=equity_holdings, 
    scope=equities_scope, 
    code=portfolio_code, 
    holdings_effective_date=holdings_effective_date, 
    instrument_identifier='Figi')

Holdings Successfully Set for Portfolio
Scope: equities_system_d93d
Code: Global-Strategies
Holdings Effective From: 2019-04-16 13:43:54.639262+00:00
Holdings Created On: 2019-04-18 13:43:56.754642+00:00



In [17]:
load_holdings(
    holdings=option_holdings, 
    scope=options_scope, 
    code=portfolio_code, 
    holdings_effective_date=holdings_effective_date, 
    instrument_identifier='Figi')

Holdings Successfully Set for Portfolio
Scope: options_system_8f3d
Code: Global-Strategies
Holdings Effective From: 2019-04-16 13:43:54.639262+00:00
Holdings Created On: 2019-04-18 13:43:58.598135+00:00



![Init](img/multiplesystems-holdings.gif)

## 5) Set your Transaction Type Configuration

Now that you have set your initial holdings you are ready to take on some transactions. Before you can add these trades to LUSID you need to set up a transaction type configuration. This allows the LUSID movement engine to correctly decompose each transaction into its underlying economic movements. Read more about configuring transaction types here [LUSID Knowledge Base: Configuring Transaction Types](https://support.lusid.com/configuring-transaction-types) and movements in LUSID here [LUSID Knowledge Base: Movements](https://support.lusid.com/what-is-a-movement-in-lusid). 

You will import the default LUSID transaction type configuration from a JSON file and then use this to set your configuration inside LUSID.

*Run the cell below to import the LUSID default transaction mapping configuration*

In [18]:
# Import the default transaction type configuration and load it into a dictionary 
default_transaction_mapping=open('data/default_transaction_mapping.json').read()
default_transaction_mapping = json.loads(default_transaction_mapping)
# Pretty print your configuration
pprint.pprint(default_transaction_mapping)

{'values': [{'aliases': [{'description': 'Purchase',
                          'transactionClass': 'Basic',
                          'transactionGroup': 'default',
                          'transactionRoles': 'LongLonger',
                          'type': 'Buy'},
                         {'description': 'PURCHASE',
                          'transactionClass': 'Basic',
                          'transactionGroup': 'alt1',
                          'transactionRoles': 'LongLonger',
                          'type': 'BY'}],
             'movements': [{'direction': 1,
                            'mappings': [],
                            'movementTypes': 'StockMovement',
                            'properties': [],
                            'side': 'Side1'},
                           {'direction': -1,
                            'mappings': [],
                            'movementTypes': 'CashCommitment',
                            'properties': [],
                            '

Now that you have imported the default configuration you can build your request to set it inside LUSID.

For further usage of the build transactions API call refer to the [LUSID API Docs: Set Configuration Transaction Types](https://docs.lusid.com/#operation/SetConfigurationTransactionTypes).

*Run the cell below to set the transaction type configuration*

In [19]:
# Initialise your list of configuration requests, one for each transaction type
configuration_requests = []

# Iterate over your configurations in the default mapping
for configuration in default_transaction_mapping['values']:
    
    # Initialise your list of aliases for this configuration
    aliases = []
    
    # Iterate over the aliases in the imported config 
    for alias in configuration['aliases']:
        # Append the alias to your list
        aliases.append(
            models.TransactionConfigurationTypeAlias(
                type=alias['type'],
                description=alias['description'],
                transaction_class=alias['transactionClass'],
                transaction_group=alias['transactionGroup'],
                transaction_roles=alias['transactionRoles']))
        
        
    # Initialise your list of movements for this configuration
    movements = []
    
    # Iterate over the movements in the impoted config
    for movement in configuration['movements']:
        
        # Add properties if they exist in the config
        if len (movement['properties']) > 0:
            key = movement['properties'][0]['key']
            value = models.PerpetualPropertyValue(movement['properties'][0]['value'])
            properties={key: value}                                    
        else:
            properties=None
            
        # Append the movement to your list
        movements.append(
            models.TransactionConfigurationMovementDataRequest(
                movement_types=movement['movementTypes'],
                side=movement['side'],
                direction=movement['direction'],
                properties=properties,
                mappings=None))
    
    # Build your configuration for this transaction type
    configuration_requests.append(
        models.TransactionConfigurationDataRequest(
        aliases=aliases,
        movements=movements,
        properties=None))

# Call LUSID to set your configuration for our transaction types
response = client.system_configuration.set_configuration_transaction_types(
    transaction_configuration_data_request=configuration_requests)

# Pretty print the response
prettyprint.transaction_type_response(response)

Transaction Configuration #1

Transaction Type Aliases
Transaction Type: Buy
Alias Description: Purchase
Transaction Class: Basic
Transaction Group: default
Transaction Roles: LongLonger


Transaction Type: BY
Alias Description: PURCHASE
Transaction Class: Basic
Transaction Group: alt1
Transaction Roles: LongLonger


Transaction Movements
Movement Types: StockMovement
Side: Side1
Direction: 1
Movement Types: CashCommitment
Side: Side2
Direction: -1



Transaction Configuration #2

Transaction Type Aliases
Transaction Type: Sell
Alias Description: Sale
Transaction Class: Basic
Transaction Group: default
Transaction Roles: LongShorter


Transaction Type: SL
Alias Description: SALE
Transaction Class: Basic
Transaction Group: alt1
Transaction Roles: LongShorter


Transaction Movements
Movement Types: StockMovement
Side: Side1
Direction: -1
Movement Types: CashCommitment
Side: Side2
Direction: 1



Transaction Configuration #3

Transaction Type Aliases
Transaction Type: CoverShort
Alias Des

![Init](img/multiplesystems-transactiontypes.gif)

## 6) Load your Transactions

Now that you have set your transaction type configuration you are ready to take on some transactions. You have made several trades over the last day that you would like to add to LUSID. You will import these from CSV files. 

*Run the cells below to import your transactions*

In [20]:
bond_transactions = pd.read_csv('data/multiplesystems-transactions-bonds.csv')
bond_transactions.head()

,PortfolioCode,InstrumentName,Quantity,Price,Currency,Figi,ClientInternal,TransactionType,TransactionId,TransactionDate,SettlementDate,SettlementAmt,TransactionToSettlementExRate,TransactionCurrency,SettlementCurrency
0,Global-Strategies,UKGiltTreasury_2.0_2025,2222.410000,NaN,GBP,NaN,imd_34534536,ACCR,1852034,NaN,NaN,2222.410000,1,GBP,GBP
1,Global-Strategies,UKGiltTreasury_3.5_2045,2552.305479,NaN,GBP,NaN,imd_54234532,ACCR,1852047,NaN,NaN,2552.305479,1,GBP,GBP
2,Global-Strategies,UKGiltTreasury_3.75_2021,6798.421233,NaN,GBP,NaN,imd_34643653,ACCR,1852071,NaN,NaN,6798.421233,1,GBP,GBP
3,Global-Strategies,UKGiltTreasury_4.5_2034,955.245205,NaN,GBP,NaN,imd_34534534,ACCR,1852078,NaN,NaN,955.245205,1,GBP,GBP
4,Global-Strategies,USTreasury_2.00_2021,7891.747945,NaN,USD,NaN,imd_34535347,ACCR,1852126,NaN,NaN,7891.747945,1,USD,USD


In [21]:
equity_transactions = pd.read_csv('data/multiplesystems-transactions-equities.csv')
equity_transactions.head()

,PortfolioCode,InstrumentName,Quantity,Price,Currency,Figi,ClientInternal,TransactionType,TransactionId,TransactionDate,SettlementDate,SettlementAmt,TransactionToSettlementExRate,TransactionCurrency,SettlementCurrency
0,Global-Strategies,Glencore_LondonStockEx_GLEN,249999.9442,NaN,GBP,BBG001MM1KV4,NaN,DIV,5788392,NaN,NaN,2.499999e+05,1,GBP,GBP
1,Global-Strategies,Kingfisher_LondonStockEx_KGF,125000.0000,2.345,GBP,BBG000BKH1W6,NaN,BUY,5788411,NaN,NaN,2.931250e+05,1,GBP,GBP
2,Global-Strategies,BurfordCapital_LondonStockEx_BUR,32050.0000,14.020,GBP,BBG000PN88Q7,NaN,SHORTSELL,5788429,NaN,NaN,4.493410e+05,1,GBP,GBP
3,Global-Strategies,EKFDiagnostics_LondonStockEx_EKF,31504.0000,0.260,GBP,BBG000BVNBN3,NaN,SELL,5788497,NaN,NaN,8.191040e+03,1,GBP,GBP
4,Global-Strategies,JustEat_LondonStockEx_JE,250000.0000,5.478,GBP,BBG0065YWM39,NaN,BUY,5788522,NaN,NaN,1.369500e+06,1,GBP,GBP


In [22]:
option_transactions = pd.read_csv('data/multiplesystems-transactions-options.csv')
option_transactions.head()

,PortfolioCode,InstrumentName,Quantity,Price,Currency,Figi,ClientInternal,TransactionType,TransactionId,TransactionDate,SettlementDate,SettlementAmt,TransactionToSettlementExRate,TransactionCurrency,SettlementCurrency
0,Global-Strategies,October 19 Calls on AMZN US,6,243.25,USD,BBG00NFXK409,NaN,CALLCONTRACT,98034324,NaN,NaN,1459.5,1,USD,USD


As each source system uses different identifiers for the transactions you will need to create a function which allows you to easily select the identifiers that you would like to use in the identifying the instrument related to each transaction.

Read more about transactions here [LUSID Knowledge Base: Transactions](https://support.lusid.com/what-is-a-transaction). 

For further usage of the upsert transactions API call refer to the [LUSID API Docs: Upsert Transactions](https://docs.lusid.com/#operation/UpsertTransactions).

*Run the cell below to create a function to handle upserting your transactions*

In [23]:
def load_transactions(
    transactions, 
    scope, 
    code, 
    trade_date, 
    settlement_date, 
    instrument_identifier,
    source_system):
    """
    This function takes a set of trades loaded from a CSV and upserts them into LUSID
    
    Keyword arguments:
    transactions (Pandas DataFrame) -- The imported transactions
    scope (string) -- The scope of the portfolio to upsert the transactions to
    code (string) -- The code of the portfolio to upsert the transactions to
    trade_date (datetime) - The timezone aware datetime at which the trade occured
    settlement_date (datetime) - The timezone aware datetime at which the trade 
    will be settled
    instrument_identifier (string) -- The name of the identifier to use to resolve
    each transaction to an instrument in the instrument master
    source_system (string) -- The source system that the transaction came from
    
    Returns:
    N/A
    """
    
    # Iterate the portfolios in the holdings CSV, note in this case you only have one
    for portfolio in transactions['PortfolioCode'].unique():
        # Initialise a list to hold your adjustments
        transactions_requests = []

        # Iterate over the holdings in each portfolio
        for index, transaction in transactions.loc[
            transactions['PortfolioCode'] == portfolio].iterrows():

            # Set your instrument identifiers based on whether or not instrument is cash
            if 'Cash' in transaction['InstrumentName']:
                identifier_key = 'Instrument/default/Currency'
                identifier = transaction['InstrumentName'].split('_')[0]
            else:
                identifier_key = 'Instrument/default/{}'.format(instrument_identifier)
                identifier = transaction[instrument_identifier]
            
            if transaction['Quantity'] == transaction['SettlementAmt']:
                price = 1
            else:
                price = transaction['Price']
            
            # Create your holding adjustment and append it to your list
            transactions_requests.append(
                models.TransactionRequest(
                    transaction_id=transaction['TransactionId'],
                    type=transaction['TransactionType'],
                    instrument_identifiers={
                        identifier_key: identifier},
                    transaction_date=trade_date,
                    settlement_date=settlement_date,
                    units=transaction['Quantity'],
                    transaction_price=models.TransactionPrice(
                        price=price,
                        type='Price'),
                    total_consideration=models.CurrencyAndAmount(
                        amount=transaction['SettlementAmt'],
                        currency=transaction['SettlementCurrency']),
                    source=source_system,
                    transaction_currency=transaction['TransactionCurrency']))

        # Call LUSID to set your initial holdings
        response = client.transaction_portfolios.upsert_transactions(
            scope=scope,
            code=code,
            transaction_request=transactions_requests)

        # Pretty print our response from LUSID
        prettyprint.transactions_response(response, scope, code)

You may have noticed that there are no transaction or settlement dates in any of the CSV files. In this case all these trades happened yesterday, so that is the trade date. For the settlement date you can make the simplifying assumption that all trades will be settling two days from the trade date.

*Run the cell below to create the trade and settlement dates*

In [24]:
trade_date = datetime.now(pytz.UTC) - timedelta(days=1)
settlement_date = datetime.now(pytz.UTC) + timedelta(days=1)
prettyprint.heading('Trade Date', str(trade_date))
prettyprint.heading('Settlement Date', str(settlement_date))

Trade Date: 2019-04-17 13:44:01.099873+00:00
Settlement Date: 2019-04-19 13:44:01.099937+00:00


Now that you have a function to upsert your transactions you can upsert them for each source. 

For bonds you will use the internal client identifier. 

Run the cell below to upsert your bond transactions for your portfolio.

*Run the cell below to upsert your bond transactions into LUSID*

In [25]:
load_transactions(
    bond_transactions, 
    bonds_scope, 
    portfolio_code, 
    trade_date, 
    settlement_date, 
    'ClientInternal',
    'Bonds')

Transactions Successfully Upserted into Portfolio
Scope: bonds_system_32cc
Code: Global-Strategies
Transactions Effective From: 2019-04-17 13:44:01.099873+00:00
Transactions Created On: 2019-04-18 13:44:01.220986+00:00



For equities you will use the FIGI identifier. 

Run the cell below to upsert your equity transactions for your portfolio.

*Run the cell below to upsert your equity transactions into LUSID*

In [26]:
load_transactions(
    equity_transactions, 
    equities_scope, 
    portfolio_code, 
    trade_date, 
    settlement_date, 
    'Figi',
    'Equities')

Transactions Successfully Upserted into Portfolio
Scope: equities_system_d93d
Code: Global-Strategies
Transactions Effective From: 2019-04-17 13:44:01.099873+00:00
Transactions Created On: 2019-04-18 13:44:03.111969+00:00



For options you will also use the FIGI identifier. 

Run the cell below to upsert your option transactions for your portfolio.

*Run the cell below to upsert your option transactions into LUSID*

In [27]:
load_transactions(
    option_transactions, 
    options_scope, 
    portfolio_code, 
    trade_date, 
    settlement_date, 
    'Figi',
    'Options')

Transactions Successfully Upserted into Portfolio
Scope: options_system_8f3d
Code: Global-Strategies
Transactions Effective From: 2019-04-17 13:44:01.099873+00:00
Transactions Created On: 2019-04-18 13:44:05.020301+00:00



![Init](img/multiplesystems-trades.gif)

## 7) Get your Holdings

You can see how these transactions have affected your holdings by trying to retrieve the holdings of your portfolio. By configuring your transaction types LUSID should have been able to decompose each transaction into its appropriate economic movements. 

For further usage of the get holdings API call refer to the [LUSID API Docs: Get Holdings](https://docs.lusid.com/#operation/GetHoldings).

*Run the cell below to get our holdings from our portfolio*

In [28]:
# Try and get your holdings, otherwise provide the error message
try:
    response = client.transaction_portfolios.get_holdings(
        scope=options_scope,
        code=portfolio_code,
        instrument_property_keys=['Instrument/default/Name'])
    prettyprint.holdings_response(response, scope, portfolio_code)
except lusid.rest.ApiException as e:
    print ('Error retrieving holdings as {}'.format(e))

Error retrieving holdings as (404)
Reason: Not Found
HTTP response headers: HTTPHeaderDict({'Date': 'Thu, 18 Apr 2019 13:44:05 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'lusid-meta-success': 'False', 'lusid-meta-duration': '172', 'lusid-meta-requestId': '0HLM389BTO6SN:00000001', 'Server': 'FINBOURNE'})
HTTP response body: {"status":404,"code":"TransactionTypeNotFound","message":"A transaction type is not configured","detailedMessage":"There is no configuration for the transaction type: 'CALLCONTRACT' from the source: 'Options'","items":[],"moreInfo":"https://docs.lusid.com/#section/Error-Codes/230"}



Here you can see that it looks like you have a transaction type in your transactions file which is not in your configuration.

![Init](img/multiplesystems-failedholdings.gif)

## 8) Update your Transaction Type Configuration

You can find out what might be the problematic transaction type by listing all of your unique transaction types for each source.

*Run the cell below to find your unique transaction types for bonds*

In [29]:
bond_transaction_types = pd.read_csv(
    'data/multiplesystems-transactions-bonds.csv')['TransactionType'].unique()
print (bond_transaction_types)

['ACCR' 'PURCHASE' 'SALE']


Here you can see that you have three transaction types in your list of bond transactions, you can check of any are in your current configuration by listing your current configuration in LUSID.

For further usage of the list configuration transaction types API call refer to the [LUSID API Docs: List Transaction Type Configuration](https://docs.lusid.com/#operation/ListConfigurationTransactionTypes).

*Run the cell below to check if any of these transaction types are configured*

In [30]:
# Call LUSID to get your transaction type configuration
response = client.system_configuration.list_configuration_transaction_types()
# Pretty print the configuration
prettyprint.transaction_type_response(response, filters=bond_transaction_types)

No matching transaction types in the configuration


It is clear from this result that none of these transaction types currently exist in the default configuration. 

So that LUSID can generate the holdings from the portfolio, you need to add the missing transaction types.

To ensure that there is no collision between the different types for each system and their underlying economic meaning, you can group the types of each source by setting the transaction group parameter. 

For your bond transactions we will set the transaction group to be 'Bonds'. Note that this is linked to the source you used when upserting the transactions. LUSID will look for any transaction types in the 'Bonds' group before falling back to the default transaction types.

*Run the cell below to add the missing transaction types for bonds into the configuration*

In [31]:
response = client.system_configuration.create_configuration_transaction_type(
    transaction_configuration_data_request = models.TransactionConfigurationDataRequest(
        aliases=[
            models.TransactionConfigurationTypeAlias(
                type='ACCR',
                description='Interest accrued on a bond',
                transaction_class='BondInstruments',
                transaction_group='Bonds',
                transaction_roles='LongLonger')
        ],
        movements=[
            models.TransactionConfigurationMovementDataRequest(
                movement_types='CashCommitment',
                side='Side2',
                direction=1,
                properties=None,
                mappings=None)
        ],
        properties=None))

# Pretty print the response
prettyprint.transaction_type_response(response, filters=['ACCR'])

response = client.system_configuration.create_configuration_transaction_type(
    transaction_configuration_data_request = models.TransactionConfigurationDataRequest(
        aliases=[
            models.TransactionConfigurationTypeAlias(
                type='PURCHASE',
                description='A purchase of a bond',
                transaction_class='BondInstruments',
                transaction_group='Bonds',
                transaction_roles='LongLonger')
        ],
        movements=[
            models.TransactionConfigurationMovementDataRequest(
                movement_types='StockMovement',
                side='Side1',
                direction=1,
                properties=None,
                mappings=None),
            models.TransactionConfigurationMovementDataRequest(
                movement_types='CashCommitment',
                side='Side2',
                direction=-1,
                properties=None,
                mappings=None)
        ],
        properties=None))

# Pretty print the response
prettyprint.transaction_type_response(response, filters=['PURCHASE'])

response = client.system_configuration.create_configuration_transaction_type(
    transaction_configuration_data_request = models.TransactionConfigurationDataRequest(
        aliases=[
            models.TransactionConfigurationTypeAlias(
                type='SALE',
                description='The sale of a bond',
                transaction_class='BondInstruments',
                transaction_group='Bonds',
                transaction_roles='LongShorter')
        ],
        movements=[
            models.TransactionConfigurationMovementDataRequest(
                movement_types='StockMovement',
                side='Side1',
                direction=-1,
                properties=None,
                mappings=None),
            models.TransactionConfigurationMovementDataRequest(
                movement_types='CashCommitment',
                side='Side2',
                direction=1,
                properties=None,
                mappings=None)
        ],
        properties=None))

# Pretty print the response
prettyprint.transaction_type_response(response, filters=['SALE'])

Transaction Configuration #17

Transaction Type Aliases
Transaction Type: ACCR
Alias Description: Interest accrued on a bond
Transaction Class: BondInstruments
Transaction Group: Bonds
Transaction Roles: LongLonger


Transaction Movements
Movement Types: CashCommitment
Side: Side2
Direction: 1



Transaction Configuration #18

Transaction Type Aliases
Transaction Type: PURCHASE
Alias Description: A purchase of a bond
Transaction Class: BondInstruments
Transaction Group: Bonds
Transaction Roles: LongLonger


Transaction Movements
Movement Types: StockMovement
Side: Side1
Direction: 1
Movement Types: CashCommitment
Side: Side2
Direction: -1



Transaction Configuration #19

Transaction Type Aliases
Transaction Type: SALE
Alias Description: The sale of a bond
Transaction Class: BondInstruments
Transaction Group: Bonds
Transaction Roles: LongShorter


Transaction Movements
Movement Types: StockMovement
Side: Side1
Direction: -1
Movement Types: CashCommitment
Side: Side2
Direction: 1





Now you can take a look at equities.

*Run the cell below to see the unique transaction types for the equity transactions*

In [32]:
equity_transaction_types = pd.read_csv(
    'data/multiplesystems-transactions-equities.csv')['TransactionType'].unique()
print (equity_transaction_types)

['DIV' 'BUY' 'SHORTSELL' 'SELL']


Now you can check to see if any of these 4 transaction types exist in the current configuration.

*Run the cell below to identify the missing transaction types for equities*

In [33]:
# Call LUSID to get your transaction type configuration
response = client.system_configuration.list_configuration_transaction_types()
# Pretty print the configuration
prettyprint.transaction_type_response(response, filters=equity_transaction_types)

No matching transaction types in the configuration


So as you can see all four types are missing from the configuration and need to be added.

*Run the cell below to add the missing types to the Equities group*

In [34]:
response = client.system_configuration.create_configuration_transaction_type(
    transaction_configuration_data_request = models.TransactionConfigurationDataRequest(
        aliases=[
            models.TransactionConfigurationTypeAlias(
                type='DIV',
                description='A dividend from an equity',
                transaction_class='EquityInstruments',
                transaction_group='Equities',
                transaction_roles='LongLonger')
        ],
        movements=[
            models.TransactionConfigurationMovementDataRequest(
                movement_types='CashCommitment',
                side='Side2',
                direction=1,
                properties=None,
                mappings=None)
        ],
        properties=None))

# Pretty print the response
prettyprint.transaction_type_response(response, filters=['DIV'])

response = client.system_configuration.create_configuration_transaction_type(
    transaction_configuration_data_request = models.TransactionConfigurationDataRequest(
        aliases=[
            models.TransactionConfigurationTypeAlias(
                type='BUY',
                description='The purchase of an equity',
                transaction_class='EquityInstrument',
                transaction_group='Equities',
                transaction_roles='LongLonger')
        ],
        movements=[
            models.TransactionConfigurationMovementDataRequest(
                movement_types='StockMovement',
                side='Side1',
                direction=1,
                properties=None,
                mappings=None),
            models.TransactionConfigurationMovementDataRequest(
                movement_types='CashCommitment',
                side='Side2',
                direction=-1,
                properties=None,
                mappings=None)
        ],
        properties=None))

# Pretty print the response
prettyprint.transaction_type_response(response, filters=['BUY'])

response = client.system_configuration.create_configuration_transaction_type(
    transaction_configuration_data_request = models.TransactionConfigurationDataRequest(
        aliases=[
            models.TransactionConfigurationTypeAlias(
                type='SHORTSELL',
                description='A short position created from borrowing and selling an equity',
                transaction_class='EquityInstruments',
                transaction_group='Equities',
                transaction_roles='ShortShorter')
        ],
        movements=[
            models.TransactionConfigurationMovementDataRequest(
                movement_types='StockMovement',
                side='Side1',
                direction=-1,
                properties=None,
                mappings=None),
            models.TransactionConfigurationMovementDataRequest(
                movement_types='CashCommitment',
                side='Side2',
                direction=1,
                properties=None,
                mappings=None)
        ],
        properties=None))

# Pretty print the response
prettyprint.transaction_type_response(response, filters=['SHORTSELL'])

response = client.system_configuration.create_configuration_transaction_type(
    transaction_configuration_data_request = models.TransactionConfigurationDataRequest(
        aliases=[
            models.TransactionConfigurationTypeAlias(
                type='SELL',
                description='A sale of an equity',
                transaction_class='EquityInstruments',
                transaction_group='Equities',
                transaction_roles='LongShorter')
        ],
        movements=[
            models.TransactionConfigurationMovementDataRequest(
                movement_types='StockMovement',
                side='Side1',
                direction=-1,
                properties=None,
                mappings=None),
            models.TransactionConfigurationMovementDataRequest(
                movement_types='CashCommitment',
                side='Side2',
                direction=1,
                properties=None,
                mappings=None)
        ],
        properties=None))

# Pretty print the response
prettyprint.transaction_type_response(response, filters=['SELL'])

Transaction Configuration #20

Transaction Type Aliases
Transaction Type: DIV
Alias Description: A dividend from an equity
Transaction Class: EquityInstruments
Transaction Group: Equities
Transaction Roles: LongLonger


Transaction Movements
Movement Types: CashCommitment
Side: Side2
Direction: 1



Transaction Configuration #21

Transaction Type Aliases
Transaction Type: BUY
Alias Description: The purchase of an equity
Transaction Class: EquityInstrument
Transaction Group: Equities
Transaction Roles: LongLonger


Transaction Movements
Movement Types: StockMovement
Side: Side1
Direction: 1
Movement Types: CashCommitment
Side: Side2
Direction: -1



Transaction Configuration #22

Transaction Type Aliases
Transaction Type: SHORTSELL
Alias Description: A short position created from borrowing and selling an equity
Transaction Class: EquityInstruments
Transaction Group: Equities
Transaction Roles: ShortShorter


Transaction Movements
Movement Types: StockMovement
Side: Side1
Direction: -1
M

In [35]:
option_transaction_types = pd.read_csv(
    'data/multiplesystems-transactions-options.csv')['TransactionType'].unique()
print (option_transaction_types)

['CALLCONTRACT']


Here you can see that there is only a single transaction type.

*Run the cell below to check if the option transaction type exists in the configuration*

In [36]:
# Call LUSID to get your transaction type configuration
response = client.system_configuration.list_configuration_transaction_types()
# Pretty print the configuration
prettyprint.transaction_type_response(response, filters=option_transaction_types)

No matching transaction types in the configuration


It looks like this is also missing from the configuration and needs to be added.

*Run the cell below to add the missing types to the Options group*

In [37]:
response = client.system_configuration.create_configuration_transaction_type(
    transaction_configuration_data_request = models.TransactionConfigurationDataRequest(
        aliases=[
            models.TransactionConfigurationTypeAlias(
                type='CALLCONTRACT',
                description='The purchase of a call options contract',
                transaction_class='EquityInstrument',
                transaction_group='Options',
                transaction_roles='LongLonger')
        ],
        movements=[
            models.TransactionConfigurationMovementDataRequest(
                movement_types='StockMovement',
                side='Side1',
                direction=1,
                properties=None,
                mappings=None),
            models.TransactionConfigurationMovementDataRequest(
                movement_types='CashCommitment',
                side='Side2',
                direction=-1,
                properties=None,
                mappings=None)
        ],
        properties=None))

# Pretty print the response
prettyprint.transaction_type_response(response, filters=['CALLCONTRACT'])

Transaction Configuration #24

Transaction Type Aliases
Transaction Type: CALLCONTRACT
Alias Description: The purchase of a call options contract
Transaction Class: EquityInstrument
Transaction Group: Options
Transaction Roles: LongLonger


Transaction Movements
Movement Types: StockMovement
Side: Side1
Direction: 1
Movement Types: CashCommitment
Side: Side2
Direction: -1





Now that you have configured the transaction types, you can try again to generate the holdings for your portfolio

*Run the cell below to try and generate the holdings for the fund based on the options source*

In [38]:
# Try and get your holdings, otherwise provide the error message
try:
    response = client.transaction_portfolios.get_holdings(
        scope=options_scope,
        code=portfolio_code,
        instrument_property_keys=['Instrument/default/Name'])
    prettyprint.holdings_response(response, scope, portfolio_code)
except lusid.rest.ApiException as e:
    print ('Error retrieving holdings as {}'.format(e))

Holdings for Portfolio
Scope: options_system_8f3d
Code: Global-Strategies

Instrument Name: October 19 Calls on AMZN US
Units: 106.0
Cost: 25987.5
Currency: USD


Instrument Name: October 19 Puts on AAPL US
Units: 150.0
Cost: 57450.0
Currency: USD


Instrument Name: September 19 Calls on BP/ LN
Units: 124.0
Cost: 17360.0
Currency: GBP


Instrument Name: USD
Units: -1459.5
Cost: -1459.5
Currency: USD
Unsettled Transaction Id: 98034324
Settlement Date: 2019-04-19 13:44:01.099937+00:00




![Init](img/multiplesystems-updatetransactiontypes.gif)

## 9) Load your End of Day Positions

Now that you are able to load your transactions into LUSID and generate the position of your fund, you need to ensure that the positions are aligned with the source systems and that there are no discrepencies. 

To do this you need to load the end of day positions into LUSID from each source system. You can use the same load holdings function you used earlier to set the initial positions.

*Run the cells below to load the end of day holdings and set them on your portfolio for each system*

In [39]:
# Make the holdings effective from two days ago
end_of_day_effective_date = datetime.now(pytz.UTC) - timedelta(days=0.5)

holdings = pd.read_csv('data/multiplesystems-holdings-eod-bonds.csv')
holdings.head()

,PortfolioCode,InstrumentName,Quantity,Price,Currency,Figi,ClientInternal
0,Global-Strategies,UKGiltTreasury_2.0_2025,405589,106.637,GBP,NaN,imd_34534536
1,Global-Strategies,UKGiltTreasury_3.5_2045,266169,134.433,GBP,NaN,imd_54234532
2,Global-Strategies,UKGiltTreasury_3.75_2021,618713,108.126,GBP,NaN,imd_34643653
3,Global-Strategies,UKGiltTreasury_4.5_2034,97481,140.572,GBP,NaN,imd_34534534
4,Global-Strategies,USTreasury_2.00_2021,1440244,97.900,USD,NaN,imd_34535347


In [40]:
load_holdings(
    holdings=holdings, 
    scope=bonds_scope, 
    code=portfolio_code, 
    holdings_effective_date=end_of_day_effective_date, 
    instrument_identifier='ClientInternal')

Holdings Successfully Set for Portfolio
Scope: bonds_system_32cc
Code: Global-Strategies
Holdings Effective From: 2019-04-18 01:44:10.632975+00:00
Holdings Created On: 2019-04-18 13:44:09.231167+00:00



In [41]:
holdings = pd.read_csv('data/multiplesystems-holdings-eod-equities.csv')
holdings.head()

,PortfolioCode,InstrumentName,Quantity,Price,Currency,Figi
0,Global-Strategies,GBP_Cash,5557333,1.0000,GBP,NaN
1,Global-Strategies,Glencore_LondonStockEx_GLEN,905141,2.7620,GBP,BBG001MM1KV4
2,Global-Strategies,Kingfisher_LondonStockEx_KGF,1487038,2.2818,GBP,BBG000BKH1W6
3,Global-Strategies,BurfordCapital_LondonStockEx_BUR,821436,14.0600,GBP,BBG000PN88Q7
4,Global-Strategies,EKFDiagnostics_LondonStockEx_EKF,894421,0.2700,GBP,BBG000BVNBN3


In [42]:
load_holdings(
    holdings=holdings, 
    scope=equities_scope, 
    code=portfolio_code, 
    holdings_effective_date=end_of_day_effective_date, 
    instrument_identifier='Figi')

Holdings Successfully Set for Portfolio
Scope: equities_system_d93d
Code: Global-Strategies
Holdings Effective From: 2019-04-18 01:44:10.632975+00:00
Holdings Created On: 2019-04-18 13:44:11.212292+00:00



In [43]:
holdings = pd.read_csv('data/multiplesystems-holdings-eod-options.csv')
holdings.head()

,PortfolioCode,InstrumentName,Quantity,Price,Currency,Figi
0,Global-Strategies,October 19 Calls on AMZN US,106,245.165094,USD,BBG00NFXK409
1,Global-Strategies,October 19 Puts on AAPL US,150,383.000000,USD,BBG00NBRV912
2,Global-Strategies,September 19 Calls on BP/ LN,124,140.000000,GBP,BBG00M2Z8958


In [44]:
load_holdings(
    holdings=holdings, 
    scope=options_scope, 
    code=portfolio_code, 
    holdings_effective_date=end_of_day_effective_date, 
    instrument_identifier='Figi')

Holdings Successfully Set for Portfolio
Scope: options_system_8f3d
Code: Global-Strategies
Holdings Effective From: 2019-04-18 01:44:10.632975+00:00
Holdings Created On: 2019-04-18 13:44:13.271131+00:00



## 10) Perform a Bi-Temporal Reconciliation 

Now you can make use of LUSID's bi-temporal data store to compare your portfolio before and after adjusting for the end of day position. You can do this by performing a bi-temporal reconciliation. Read more about reconciliations here [LUSID Knowledge Base: Performing a Reconciliation](https://support.lusid.com/how-do-i-reconcile-portfolios-from-different).

By looking for reconciliation breaks between your portfolio before it was adjusted and after it was adjusted you can see what discrepancies have arisen from any issues with the transaction feed during the day. 

For further usage of the reconcile holdings API call refer to the [LUSID API Docs: Reconcile Holdings](https://docs.lusid.com/#operation/ReconcileValuation).

First you can check that the positions in LUSID are aligned with the bond source system.

*Run the cell below to reconcile your portfolio in the bonds scope with itself before and after the adjustment*

In [45]:
# Build your left side of the reconciliation before you made the adjustment
before_adjustment = models.PortfolioReconciliationRequest(
    portfolio_id=models.ResourceId(
        scope=bonds_scope,
        code=portfolio_code),
    effective_at=end_of_day_effective_date - timedelta(hours=1),
    as_at=datetime.now(pytz.UTC).isoformat())

# Build your right side of the reconciliation after you made the adjustment
after_adjustment = models.PortfolioReconciliationRequest(
    portfolio_id=models.ResourceId(
        scope=bonds_scope,
        code=portfolio_code),
    effective_at=end_of_day_effective_date + timedelta(hours=1),
    as_at=datetime.now(pytz.UTC).isoformat())

# Create your reconciliation request
reconcile_holdings_request = models.PortfoliosReconciliationRequest(
    left=before_adjustment,
    right=after_adjustment,
    instrument_property_keys=['Instrument/default/Name', 'Instrument/default/Figi'])

# Reconcile your two portfolios
response = client.reconciliations.reconcile_holdings(
    portfolios_reconciliation_request=reconcile_holdings_request)

# Pretty print the response
prettyprint.reconciliation_response(
    response,
    bonds_scope,
    portfolio_code)

Reconciliation Breaks for Portfolio
Scope: bonds_system_32cc
Code: Global-Strategies

No Reconciliation Breaks


Next you can look at equities.

*Run the cell below to reconcile your portfolio in the equities scope with itself before and after the adjustment*

In [46]:
# Build your left side of the reconciliation before you made the adjustment
before_adjustment = models.PortfolioReconciliationRequest(
    portfolio_id=models.ResourceId(
        scope=equities_scope,
        code=portfolio_code),
    effective_at=end_of_day_effective_date - timedelta(hours=1),
    as_at=datetime.now(pytz.UTC).isoformat())

# Build your right side of the reconciliation after you made the adjustment
after_adjustment = models.PortfolioReconciliationRequest(
    portfolio_id=models.ResourceId(
        scope=equities_scope,
        code=portfolio_code),
    effective_at=end_of_day_effective_date + timedelta(hours=1),
    as_at=datetime.now(pytz.UTC).isoformat())

# Create your reconciliation request
reconcile_holdings_request = models.PortfoliosReconciliationRequest(
    left=before_adjustment,
    right=after_adjustment,
    instrument_property_keys=['Instrument/default/Name', 'Instrument/default/Figi'])

# Reconcile your two portfolios
response = client.reconciliations.reconcile_holdings(
    portfolios_reconciliation_request=reconcile_holdings_request)

# Pretty print the response
prettyprint.reconciliation_response(
    response,
    equities_scope,
    portfolio_code)

Reconciliation Breaks for Portfolio
Scope: equities_system_d93d
Code: Global-Strategies

No Reconciliation Breaks


Finally you can look at options.

*Run the cell below to reconcile your portfolio in the options scope with itself before and after the adjustment*

In [47]:
# Build your left side of the reconciliation before you made the adjustment
before_adjustment = models.PortfolioReconciliationRequest(
    portfolio_id=models.ResourceId(
        scope=options_scope,
        code=portfolio_code),
    effective_at=end_of_day_effective_date - timedelta(hours=1),
    as_at=datetime.now(pytz.UTC).isoformat())

# Build your right side of the reconciliation after you made the adjustment
after_adjustment = models.PortfolioReconciliationRequest(
    portfolio_id=models.ResourceId(
        scope=options_scope,
        code=portfolio_code),
    effective_at=end_of_day_effective_date + timedelta(hours=1),
    as_at=datetime.now(pytz.UTC).isoformat())

# Create your reconciliation request
reconcile_holdings_request = models.PortfoliosReconciliationRequest(
    left=before_adjustment,
    right=after_adjustment,
    instrument_property_keys=['Instrument/default/Name', 'Instrument/default/Figi'])

# Reconcile your two portfolios
response = client.reconciliations.reconcile_holdings(
    portfolios_reconciliation_request=reconcile_holdings_request)

# Pretty print the response
prettyprint.reconciliation_response(
    response,
    options_scope,
    portfolio_code)

Reconciliation Breaks for Portfolio
Scope: options_system_8f3d
Code: Global-Strategies

No Reconciliation Breaks


![Init](img/multiplesystems-reconciliation.gif)

## 11) Group the Sources to Create an Overall View of your Fund

Now that you've ensured that the holdings in LUSID match those in the source systems, you can be confident that LUSID has correctly ingested and processed the day's transactions. 

So far your fund has been split up by the different source systems. To get an overall view of your fund you can now combine the portfolio in the bonds scope, equities scope and options scope. 

To do this you can make use of portfolio groups.

Read more about portfolio groups here [LUSID Knowledge Base: How do you Group and Aggregate Portfolios?](https://support.lusid.com/how-do-you-group-and-aggregate-portfolios)

*Run the cell below to create a new scope and a portfolio group for the combined fund*

In [48]:
fund_scope = 'combined_systems_{}'.format(str(uuid.uuid4())[:4])

group_request = models.CreatePortfolioGroupRequest(
    id=portfolio_code+'Group',
    display_name=portfolio_code + 'Group')

response = client.portfolio_groups.create_portfolio_group(
    scope=fund_scope,
    create_portfolio_group_request=group_request)

prettyprint.portfolio_group_response(response, 'created')

Portfolio Group Created
Name: Global-StrategiesGroup
Scope: combined_systems_7a30
Code: Global-StrategiesGroup
Portfolios Inside Group: 




You can now add the portfolio from each scope to the group.

*Run the cell below to add each of the portfolios representing the different source systems into a single group*

In [49]:
for scope in scopes:
    
    response = client.portfolio_groups.add_portfolio_to_group(
        scope=fund_scope,
        code=portfolio_code+'Group',
        resource_id=models.ResourceId(
            scope=scope,
            code=portfolio_code))

    prettyprint.get_portfolio_group_response(response)

Portfolio Group: 
Name: Global-StrategiesGroup
Scope: combined_systems_7a30
Code: Global-StrategiesGroup
Portfolios Inside Group: 
Global-Strategies
Subgroups Inside Group: 


Portfolio Group: 
Name: Global-StrategiesGroup
Scope: combined_systems_7a30
Code: Global-StrategiesGroup
Portfolios Inside Group: 
Global-Strategies
Global-Strategies
Subgroups Inside Group: 


Portfolio Group: 
Name: Global-StrategiesGroup
Scope: combined_systems_7a30
Code: Global-StrategiesGroup
Portfolios Inside Group: 
Global-Strategies
Global-Strategies
Global-Strategies
Subgroups Inside Group: 




## 12) Value your Fund

In [50]:
prices = pd.read_csv('data/multiplesystems-prices.csv')
prices.head()

,instrument_name,currency,figi,price_current,ticker,isin,sedol,client_internal
0,UKGiltTreasury_2.0_2025,GBP,NaN,106.637,UKT 2 09/07/25,GB00BTHH2R79,NaN,imd_34534536
1,UKGiltTreasury_3.5_2045,GBP,NaN,134.433,UKT 3.5 01/22/45,GB00BN65R313,NaN,imd_54234532
2,UKGiltTreasury_3.75_2021,GBP,NaN,108.126,UKT 3.75 09/07/21,GB00B4RMG977,NaN,imd_34643653
3,UKGiltTreasury_4.5_2034,GBP,NaN,140.572,UKT 4.5 09/07/34,GB00B52WS153,NaN,imd_34534534
4,USTreasury_2.00_2021,USD,NaN,97.900,T 2 12/31/21,US912828U816,NaN,imd_34535347


In [51]:
instrument_quotes = []

for index, quote in prices.iterrows():

    if type(quote['figi']) is str:
        instrument_id = quote['figi']
        instrument_id_type = 'Figi'
    elif type(quote['isin']) is str:
        instrument_id = quote['isin']
        instrument_id_type = 'Isin'
    else:
        instrument_id = quote['sedol']
        instrument_id_type = 'Sedol'
    
    instrument_quotes.append(models.UpsertQuoteRequest(
        quote_id=models.QuoteId(
            provider='DataScope',
            price_source='USDRC',
            instrument_id=instrument_id,
            instrument_id_type=instrument_id_type,
            quote_type='Price',
            price_side='Mid'),
        metric_value=models.MetricValue(
            value=quote['price_current'],
            unit=quote['currency']),
        effective_at=holdings_effective_date,
        lineage='InternalSystem'
    ))
    
response = client.quotes.upsert_quotes(
    scope=fund_scope,
    upsert_quote_request=instrument_quotes)

print (response)

{'as_at': datetime.datetime(2019, 4, 18, 13, 44, 15, 227271, tzinfo=tzlocal()),
 'links': None}


In [52]:
inline_recipe = models.ConfigurationRecipe(
    code='quotes_recipe',
    market=models.MarketContext(
        market_rules=[
            models.MarketDataKeyRule(
               key='Equity.Figi.*',
               supplier='DataScope',
               data_scope=fund_scope,
               quote_type='Price',
               price_side='Mid'),
           models.MarketDataKeyRule(
               key='Equity.Isin.*',
               supplier='DataScope',
               data_scope=fund_scope,
               quote_type='Price',
               price_side='Mid'),
            models.MarketDataKeyRule(
               key='Fx.CurrencyPair.*',
               supplier='DataScope',
               data_scope=fund_scope,
               quote_type='Rate',
               price_side='Mid')
        ],
        suppliers=models.MarketContextSuppliers(
            commodity='DataScope',
            credit='DataScope',
            equity='DataScope',
            fx='DataScope',
            rates='DataScope'),
        options=models.MarketOptions(
            default_supplier='DataScope',
            default_instrument_code_type='Isin',
            default_scope=fund_scope)
    )
)

aggregation_request = models.AggregationRequest(
    inline_recipe=inline_recipe,
    effective_at=end_of_day_effective_date,
    metrics=[
        models.AggregateSpec(key='Holding/default/SubHoldingKey',
        op='Value'),
        models.AggregateSpec(key='Instrument/default/Name',
        op='Value'),
        models.AggregateSpec(key='Holding/default/Units',
        op='Sum'),
        models.AggregateSpec(key='Holding/default/Cost',
        op='Sum'),
        models.AggregateSpec(key='Holding/default/PV',
        op='Sum'),
    ],
    group_by=[
        'Holding/default/SubHoldingKey'
    ])

response = client.aggregation.get_aggregation_by_group(
    scope=fund_scope,
    code=portfolio_code+'Group',
    aggregation_request=aggregation_request)

print (response)

ApiException: (404)
Reason: Not Found
HTTP response headers: HTTPHeaderDict({'Date': 'Thu, 18 Apr 2019 13:44:17 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'lusid-meta-success': 'False', 'lusid-meta-duration': '1689', 'lusid-meta-requestId': '0HLM389BTO6TF:00000001', 'Server': 'FINBOURNE'})
HTTP response body: {"status":404,"code":"PriceNotFound","message":"Failed to find or create market data item [Analytic LUID_CZ7TP760].","detailedMessage":"Failed to find or create market data item [Analytic LUID_CZ7TP760]. This occurred because LUID_CZ7TP760 is not in the loaded Analytics set. Implies incorrect dependencies or missing data for time.","items":[],"moreInfo":"https://docs.lusid.com/#section/Error-Codes/189"}
